In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, SQLTransformer
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StructType

In [ ]:
streaming_path = 'inputStreaming/'
model_path = 'pipelineClassificationModel/'

output_path = 'predictions/'

In [ ]:
# Define the schema of the input data
inputSchema = StructType()\
.add("sepallength", "double")\
.add("sepalwidth", "double")\
.add("petallength", "double")\
.add("petalwidth", "double")\
.add("variety","double")

In [ ]:
# Define input stream
data_stream = spark.readStream\
                    .format('csv')\
                    .option('path', streaming_path)\
                    .option('header', True)\
                    .schema(inputSchema)\
                    .load()

In [ ]:
data_stream.printSchema()

In [ ]:
# Real ML model
model = PipelineModel.load(model_path)

In [ ]:
predictions = model.transform(data_stream)

In [ ]:
predictions.printSchema()

In [ ]:
predictionStreamWriter = predictions\
.select('sepallength', 'sepalwidth', 'petallength', 'petalwidth', 'prediction')\
.writeStream\
.format("csv")\
.outputMode('append')\
.option("path", output_path)\
.option("checkpointLocation", "checkpoint/")

In [ ]:
prediction_stream = predictionStreamWriter.start()

In [ ]:
prediction_stream.stop()